# Cross-response functions

$$ r_{i} (t,\tau) = \log m_{i}(t + \tau) - \log m_{i}(t) = \log \frac{m_{i}(t + \tau)}{m_{i}(t)} $$

In [1]:
# Import modules

from matplotlib import pyplot as plt
import numpy as np
import pickle

%matplotlib inline

In [2]:
ticker = 'AAPL'
day = '09'

In [3]:
def log_return(midpoint, t, tau):
    return np.log(midpoint[t + tau] / midpoint[t])

In [4]:
# Load data
midpoint = pickle.load(open('../Data/midpoint_data/midpoint_201603%s_%s.pickl' % (day,ticker), 'rb'))
time = pickle.load(open('../Data/midpoint_data/time_201603%s_%s.pickl' % (day,ticker), 'rb'))

In [5]:
log_return = 0. * time
tau = 1000

In [ ]:
plt.figure(figsize=(16,9))
plt.plot(time, log_return)

In [9]:
for tau in range(1000, len(time), 1000):
    print(tau)
    for t_idx in range(0, len(time), tau):
        if (t_idx + tau < len(time)):
            log_return[t_idx] = np.log(midpoint[t_idx + tau] / midpoint[t_idx])

    pickle.dump(log_return, open('../Data/test_data/cross_response_%s_%s_%s.pickl' %(ticker, day, tau), 'wb'))
    


1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
143000
144000
145000
146000
147000
148000
149000
150000
151000
152000
153000
154000
155000
156000
157000
158000
15

OSError: [Errno 28] No space left on device